# Exploration des Données pour l'ETF Personnalisé

Ce notebook réalise une exploration des données financières pour la construction d'un ETF personnalisé ciblant les marchés émergents et les technologies.

## Objectifs
- Explorer les données de marché des actifs potentiels pour l'ETF
- Analyser les caractéristiques fondamentales des actifs
- Examiner les métriques de liquidité
- Visualiser les corrélations entre actifs

In [ ]:
# Imports nécessaires
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from pathlib import Path
import yaml
import json
from datetime import datetime, timedelta

# Configuration pour les visualisations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("viridis")

# Définir les chemins de données
data_dir = Path("../data")
raw_dir = data_dir / "raw"
processed_dir = data_dir / "processed"
config_path = Path("../config/config.yaml")

# Créer les répertoires s'ils n'existent pas
raw_dir.mkdir(parents=True, exist_ok=True)
processed_dir.mkdir(parents=True, exist_ok=True)

## Chargement de la Configuration

Commençons par charger la configuration du projet depuis le fichier YAML.

In [ ]:
# Charger la configuration
try:
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
        print("Configuration chargée avec succès!")
except Exception as e:
    print(f"Erreur lors du chargement de la configuration: {e}")
    # Créer une configuration par défaut si le fichier n'existe pas
    config = {
        "etf": {
            "name": "Innovation Tech Émergente ETF",
            "benchmark": "MSCI Emerging Markets Technology Index"
        },
        "asset_selection": {
            "markets": ["Brazil", "Russia", "India", "China", "South Africa"],
            "sectors": ["Technology", "Telecommunications", "Consumer Electronics"]
        },
        "backtesting": {
            "start_date": "2018-01-01",
            "end_date": "2022-12-31"
        }
    }

# Afficher les informations de base sur l'ETF
print(f"Nom de l'ETF: {config['etf']['name']}")
print(f"Indice de référence: {config['etf']['benchmark']}")
print(f"\nMarchés ciblés: {', '.join(config['asset_selection']['markets'])}")
print(f"Secteurs ciblés: {', '.join(config['asset_selection']['sectors'])}")
print(f"\nPériode de backtest: {config['backtesting']['start_date']} à {config['backtesting']['end_date']}")

## Collecte des Données de Marché

Pour notre ETF, nous avons besoin de données de marché pour les actifs des pays émergents ciblés. Commençons par identifier les ETFs les plus connus pour ces marchés afin d'extraire leurs composants.

In [ ]:
# Définir les ETFs représentatifs pour chaque marché
market_etfs = {
    "Brazil": "EWZ",
    "Russia": "RSX",
    "India": "INDA",
    "China": "MCHI",
    "South Africa": "EZA",
    "Mexico": "EWW",
    "Indonesia": "EIDO",
    "Turkey": "TUR"
}